In [10]:
import pandas as pd
import json, requests
import os
from dotenv import load_dotenv
load_dotenv()  

True

In [11]:
MOVIES_DF = pd.read_csv("../data/cleaned/final_df.csv")

In [12]:
MOVIES_DF.head().imdb_id

0    tt1179933
1    tt0360323
2    tt1549821
3    tt1609921
4    tt1160368
Name: imdb_id, dtype: object

In [13]:
MOVIES_DF['date'] = MOVIES_DF['release_date'].apply(lambda x: pd.to_datetime(x, errors='coerce'))

In [14]:
ids = MOVIES_DF.sort_values(['date'], ascending=False)['imdb_id'].tolist()
print(f"Found {len(ids)} movie IDs to fetch.")

# Create batches of 1000 ids
batch_size = 1000
batches = [ids[i:i + batch_size] for i in range(0, len(ids), batch_size)]
print(f"Created {len(batches)} batches of up to {batch_size} IDs each.")


Found 10067 movie IDs to fetch.
Created 11 batches of up to 1000 IDs each.


In [15]:
MOVIES_DF[MOVIES_DF.date > '1990-01-01']

,ticker,date,title,distributor,gross,percent_yd,percent_lw,theaters,per_theater,total_gross,...,vote_average,vote_count,origin_country,spoken_languages,genre_ids,genre_names,production_company_ids,production_company_names,belongs_to_collection,gross_per_theater
0,PARA,2016-03-10,10 Cloverfield Lane,Paramount Pi…,11414,0.32,-0.12,120.0,95.0,72082999,...,7.000,8351.0,US,English,"53, 878, 18, 27","Thriller, Science Fiction, Drama, Horror",11461,Bad Robot,NaN,95.116667
1,Private,2006-08-17,10th & Wolf,ThinkFilm,1791,0.00,0.00,6.0,299.0,49783,...,5.856,108.0,US,English,"28, 80, 18, 9648, 53","Action, Crime, Drama, Mystery, Thriller",41427,Suzanne DeLaurentiis Productions,NaN,298.500000
2,6758,2009-03-03,12,Sony Picture…,344,0.00,0.00,5.0,69.0,119587,...,5.600,50.0,US,English,18,Drama,NaN,NaN,NaN,68.800000
3,6758,2009-03-03,12,Sony Picture…,344,0.00,0.00,5.0,69.0,119587,...,5.000,57.0,US,English,35,Comedy,NaN,NaN,NaN,68.800000
4,DIS,2009-03-26,12 Rounds,20th Century…,4832,0.00,0.98,29.0,167.0,12187944,...,5.904,819.0,US,English,"28, 53, 80","Action, Thriller, Crime","1557, 17887, 2890, 10339","The Mark Gordon Company, Midnight Sun Pictures...",12 Rounds Collection,166.620690
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10062,SONY,2011-07-07,Zookeeper,Sony Pictures,13006,-0.32,-0.57,94.0,138.0,80360866,...,5.428,1730.0,US,English,"35, 10749, 10751","Comedy, Romance, Family","8000, 46222, 878, 5, 21","Broken Road Productions, Hey Eddie, Happy Madi...",NaN,138.361702
10063,PARA,2001-09-27,Zoolander,Paramount Pi…,1430000,0.49,-0.38,2285.0,626.0,39272000,...,6.207,4697.0,US,English,35,Comedy,"4, 79, 268652, 172, 258, 2932","Paramount Pictures, Village Roadshow Pictures,...",Zoolander Collection,625.820569
10064,PARA,2016-02-11,Zoolander 2,Paramount Pi…,2649,-0.21,-0.74,60.0,44.0,28848693,...,4.921,2306.0,US,English,"35, 12, 28","Comedy, Adventure, Action","4, 2932, 258","Paramount Pictures, Red Hour, Scott Rudin Prod...",Zoolander Collection,44.150000
10065,SONY,2006-08-10,Zoom,Sony Pictures,94312,0.00,0.00,303.0,311.0,11406340,...,5.306,624.0,US,English,"10751, 35, 12, 28, 878","Family, Comedy, Adventure, Action, Science Fic...","497, 598, 12626","Revolution Studios, Team Todd, Boxing Cat Films",NaN,311.260726


In [16]:
from utils import query_omdb, get_movies_from_ids

## Extract Movie Details from OMDB API



In [20]:
OMDB_API_KEY = os.getenv("OMDB_API_KEY")

Note that there are a bunch of different endpoints for different sorts of info, meaning we could query by:

View API Reference here: [OMDB API Reference](www.omdbapi.com)

In [8]:
example_movie = query_omdb("tt0111161", OMDB_API_KEY)

In [8]:
example_movie

{'Title': 'The Shawshank Redemption',
 'Year': '1994',
 'Rated': 'R',
 'Released': '14 Oct 1994',
 'Runtime': '142 min',
 'Genre': 'Drama',
 'Director': 'Frank Darabont',
 'Writer': 'Stephen King, Frank Darabont',
 'Actors': 'Tim Robbins, Morgan Freeman, Bob Gunton',
 'Plot': 'A banker convicted of uxoricide forms a friendship over a quarter century with a hardened convict, while maintaining his innocence and trying to remain hopeful through simple compassion.',
 'Language': 'English',
 'Country': 'United States',
 'Awards': 'Nominated for 7 Oscars. 21 wins & 42 nominations total',
 'Poster': 'https://m.media-amazon.com/images/M/MV5BMDAyY2FhYjctNDc5OS00MDNlLThiMGUtY2UxYWVkNGY2ZjljXkEyXkFqcGc@._V1_SX300.jpg',
 'Metascore': '82',
 'imdbRating': '9.3',
 'imdbVotes': '3,120,594',
 'imdbID': 'tt0111161',
 'Type': 'movie',
 'DVD': 'N/A',
 'BoxOffice': '$28,767,189',
 'Production': 'N/A',
 'Website': 'N/A',
 'Response': 'True',
 'Rating_InternetMovieDatabase': '9.3/10',
 'Rating_RottenTomatoe

### Collecting Data batches

In [18]:
collect = True
all_movies = []

In [ ]:
if collect:
    batch_1 = get_movies_from_ids(OMDB_API_KEY, batches[0])
    print(f"Retrieved {len(batch_1)} movies from batch 1")
    all_movies.extend(batch_1)
    print(f"Total movies so far: {len(all_movies)}")

    # Save progress
[    pd.DataFrame(all_movies).to_csv("movies_batch_1.csv", index=False)
]    

In [24]:
out =pd.DataFrame(all_movies)

In [25]:
out

""
